In [ ]:
import requests
import tarfile
import boto3
import os
import random
import string
from dotenv import load_dotenv
from botocore.exceptions import ClientError
import time

import 



In [ ]:
# Load environment variables
load_dotenv('path/to/your/.env')

def create_s3_bucket_name(base_string):
    random_string = ''.join(random.choices(string.digits, k=10))
    return f"{base_string}-{random_string}"

def create_s3_bucket(bucket_name):
    try:
        s3 = boto3.client('s3', region_name=os.environ.get('AWS_DEFAULT_REGION'))
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': os.environ.get('AWS_DEFAULT_REGION')})
        print(f"Bucket {bucket_name} created.")
    except boto3.exceptions.Boto3Error as e:
        print(f"Failed to create bucket: {e}")

def retry_upload(local_path, bucket_name, s3_path, retries=3, delay=2):
    s3 = boto3.client('s3', region_name=os.environ.get('AWS_DEFAULT_REGION'))
    for attempt in range(retries):
        try:
            s3.upload_file(local_path, bucket_name, s3_path)
            print(f"Uploaded: {s3_path}")
            return
        except ClientError as e:
            print(f"Retry {attempt+1}/{retries} failed for {local_path}. Error: {e}")
            time.sleep(delay)
    print(f"Failed to upload {local_path} after {retries} retries.")

def upload_directory_to_s3(bucket_name, directory_path):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            try:
                local_path = os.path.join(root, file)
                s3_path = os.path.relpath(local_path, start=directory_path)
                retry_upload(local_path, bucket_name, s3_path)
            except Exception as e:
                print(f"Failed to upload {local_path}. Error: {e}")

# URL of the Stanford Dogs Dataset
url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'

# Download the dataset
try:
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    tar_file_path = 'stanford_dogs_dataset.tar'
    with open(tar_file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
except requests.exceptions.RequestException as e:
    print(f"Error downloading dataset: {e}")
    exit(1)

# Decompress the tar file
try:
    print("Decompressing the dataset...")
    with tarfile.open(tar_file_path, 'r') as tar:
        tar.extractall(path='stanford_dogs_dataset')
    print('Download and extraction complete!')
except (tarfile.TarError, IOError) as e:
    print(f"Error extracting tar file: {e}")
    exit(1)

# Create S3 Bucket
base_bucket_name = 'stanford-dogs-dataset'
bucket_name = create_s3_bucket_name(base_bucket_name)
create_s3_bucket(bucket_name)

# Save the bucket name in the .env file
with open('path/to/your/.env', 'a') as file:
    file.write(f"\nMY_APP_BUCKET_NAME={bucket_name}")

# Upload to S3
directory_path = 'stanford_dogs_dataset'  # Local directory with the dataset
print("Uploading to S3...")
upload_directory_to_s3(bucket_name, directory_path)
print("Upload complete!")


In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('path/to/your/.env')

# Initialize the SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Retrieve the bucket name from the environment variable
bucket_name = os.environ.get('MY_APP_BUCKET_NAME')
data_uri = f's3://{bucket_name}/stanford_dogs_dataset'

# Define the SageMaker estimator
try:
    estimator = PyTorch(entry_point='vggnet_train.py',  # Replace with your training script
                        role=role,
                        framework_version='1.8.0',
                        py_version='py3',
                        instance_count=1,
                        instance_type='ml.p3.2xlarge',
                        hyperparameters={
                            'model_type': 'vgg',
                            'epochs': 10
                        })

    # Hyperparameter tuning
    hyperparameter_ranges = {
        'learning_rate': ContinuousParameter(0.001, 0.1),
        'batch_size': IntegerParameter(32, 256),
    }

    objective_metric_name = 'validation:accuracy'

    tuner = HyperparameterTuner(estimator,
                                objective_metric_name,
                                hyperparameter_ranges,
                                max_jobs=20,
                                max_parallel_jobs=3)

    # Start hyperparameter tuning
    tuner.fit({'training': data_uri})

    # Deploy the best model
    best_model = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

    # Optionally update the .env file with the SageMaker instance name
    with open('path/to/your/.env', 'a') as file:
        file.write(f"\nMY_APP_SAGEMAKER_INSTANCE_NAME={best_model.endpoint_name}")

except boto3.exceptions.Boto3Error as e:
    print(f"Error in SageMaker operation: {e}")
